In [ ]:
import sys
!{sys.executable} -m pip install 'git+https://github.com/gaetanochiriaco/food-recognition'

In [2]:
import torch
import zipfile
import os
import timm

In [3]:
from food_recognition.dataloader import load_data
from food_recognition.train import testing_loop

In [4]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [5]:
!tar -xf /content/drive/MyDrive/TesiMagistrale/data/Food2k_resize.tar -C /content

In [6]:
if not os.path.exists("/content/Food2k_label"):
  with zipfile.ZipFile('/content/drive/MyDrive/TesiMagistrale/data/Food2k_label.zip', "r") as z:
    z.extractall("/content/")

In [7]:
batch_size = 16
load_check = True
mean_img = [0.5457954, 0.44430383, 0.34424934]
sd_img = [0.23273608, 0.24383051, 0.24237761]
num_classes = 2000
save_path = "/content/drive/MyDrive/TesiMagistrale/VIT/checkpoints"
model_name = "swin_base_finetune"

In [8]:
model = timm.create_model('swin_base_patch4_window7_224_in22k', pretrained=True, num_classes=num_classes)

/usr/local/lib/python3.10/dist-packages/timm/models/_factory.py:114: UserWarning: Mapping deprecated model name swin_base_patch4_window7_224_in22k to current swin_base_patch4_window7_224.ms_in22k.
  model = create_fn(
/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [9]:
train_dataset, train_loader, test_dataset, test_loader = load_data(
    image_path= "/content/Food2k_resize",
    train_dir = "/content/Food2k_label/train_finetune.txt" ,
    test_dir = "/content/Food2k_label/test_finetune.txt",
    batch_size= batch_size,
    mean_img = mean_img,
    sd_img = sd_img
)

In [10]:
model = model.to(memory_format=torch.channels_last)
model = model.cuda()

In [11]:
list_checkpoint = os.path.exists(save_path+f"/{model_name}.pt")
if load_check and list_checkpoint:
  checkpoint = torch.load(save_path+f"/{model_name}.pt")
  model.load_state_dict(checkpoint['model_state_dict'])


In [ ]:
testing_loop(model,
             test_loader)